## Experimental Settings

In [1]:
username = "" # Username of user on server
exp_name = "" # Name of Experiment on TestBed
proj_name = "" # Name of Project on TestBed
default_node = "" # Default node to run the experiment from
ds = ""
host_ip = ""
host_servers_ip = ""
default_ip = "{}.{}.{}.{}".format(default_node, exp_name, proj_name, host_servers_ip)

# change into the correct directory
proj_path = "~/DSEF/{}".format(ds)
%cd '$proj_path'

/home/sgt43/DSEF


## Imports / Definitions

In [ ]:
import yaml
import os
from importlib import reload
from altair import *
from altair_widgets import interact_with
import pandas as pd
import pydsef
from pydsef import Experiment, DSEFLoader, util

%load_ext yamlmagic
#TODO: do this in Jupyter config

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [ ]:
%%yaml conf --loader DSEFLoader
client_types:
    closed: &closed client_closed
    open: &open client_open
benchmark:
    rw: &rw rw_benchmark
    tpca: &tpca tpca
    
experiment:
    duration: 10
    name: dsef
    clients: !include [1, 2, 4]
    server_timeout: 30
    servers: !include [2]
    benchs: !include [*tpcc]
    num_cores: 2
    client_type: *closed
    
host:
    node-1: node-1
    node-2: node-2
    node-3: node-3
    node-4: node-4
    node-5: node-5

In [ ]:
exp = Experiment(default_ip, username, ds, conf)
pd.DataFrame(exp.experiment_list)

# Experiment
Launch Instances of the Distributed System
    
Setup Benchmark on Clients
    
Start the Experiment
    
Kill all Instances of the Distributed System 

## Loop

In [ ]:
exp.set_executable('run.py')
# exp.push_files(files)
exp.set_archive('log')

# exp.show_log = False
# exp.save_log = True
res = exp.run()

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
interact_with(df.transpose())